# Contains Neural Network and IID Cross-Validation

In [108]:
#calculate Baseline Score!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [109]:
import numpy as np
import pandas as pd
import copy

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from GetCViterable import GetCViterable

In [110]:
#reading dataframe
df = pd.read_csv("dataset/phase_3_TRAIN_7d499bff69ca69b6_6372c3e_MLPC2021_generic.csv")

In [111]:
# drop target value, student annotations and string ID from input features:
X = df.drop(columns=['quadrant','mean_A','mean_V','id','score_mode','score_key_strength'])

# we want to predict the quadrant:
y = df['quadrant'].values

In [112]:
from GetCViterable import GetCViterable
cvIterable=GetCViterable().iterable("dataset/phase_3_TRAIN_7d499bff69ca69b6_6372c3e_MLPC2021_generic.csv")


[   0    1    2 ... 2604 2605 2606]
[   0    1    2 ... 2604 2605 2606]
[   0    1    2 ... 2604 2605 2606]
[   0    1    2 ... 2604 2605 2606]
[   0    1    2 ... 2604 2605 2606]
[   0    1    2 ... 2604 2605 2606]
[   0    1    2 ... 2604 2605 2606]
[   0    1    2 ... 2604 2605 2606]


In [116]:
clf=GridSearchCV(MLPClassifier(),param_grid={} , cv=cvIterable)
clf.fit(X,y)

GridSearchCV(cv=[(array([   0,    1,    2, ..., 2604, 2605, 2606]),
                  array([1739, 1740, 1741, 1742, 1743, 1744, 1745, 1746, 1747, 1748, 1749,
       1750, 1751, 1752, 1753, 1754, 1755, 1756, 1757, 1758, 1759, 1760,
       1761, 1762, 1763, 1764, 1765, 1766, 1767, 1768, 1769, 1770, 1771,
       1772, 1773, 1774, 1775, 1776, 1777, 1778, 1779, 1780, 1781, 1782,
       1783, 1784, 1785, 1786, 1787, 1788, 1789, 1790, 1791, 1792, 1793,
       1794, 1795, 1796, 1...
                                     learning_rate_init=0.001, max_fun=15000,
                                     max_iter=200, momentum=0.9,
                                     n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_state=None, shuffle=True,
                                     solver='adam', tol=0.0001,
                                     validation_fraction=0.1, verbose=False,
                                

In [117]:
clf.cv_results_

{'mean_fit_time': array([15.82355174]),
 'std_fit_time': array([10.67596426]),
 'mean_score_time': array([0.01067975]),
 'std_score_time': array([0.00299303]),
 'params': [{}],
 'split0_test_score': array([0.45670225]),
 'split1_test_score': array([0.60630841]),
 'split2_test_score': array([0.63325991]),
 'split3_test_score': array([0.51111111]),
 'split4_test_score': array([0.55672823]),
 'split5_test_score': array([0.70094937]),
 'split6_test_score': array([0.56719818]),
 'split7_test_score': array([0.74688797]),
 'mean_test_score': array([0.59739318]),
 'std_test_score': array([0.08961193]),
 'rank_test_score': array([1])}